In [1]:
# Load necessar libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import numpy as np
import re


In [2]:
#Loading the heart_disease_uci dataset
train_data = pd.read_csv('../../data/breast_cancer/breast_cancer_data_train.csv')
test_data = pd.read_csv('../../data/breast_cancer/breast_cancer_data_test.csv')
train_data.head()



,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,859471,9.029,17.33,58.79,250.5,0.10660,0.14130,0.31300,0.04375,0.2111,...,22.65,65.50,324.7,0.14820,0.43650,1.25200,0.17500,0.4228,0.11750,0
1,873593,21.090,26.57,142.70,1311.0,0.11410,0.28320,0.24870,0.14960,0.2395,...,33.48,176.50,2089.0,0.14910,0.75840,0.67800,0.29030,0.4098,0.12840,1
2,859196,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,...,19.23,65.59,310.1,0.09836,0.16780,0.13970,0.05087,0.3282,0.08490,0
3,88466802,10.650,25.22,68.01,347.0,0.09657,0.07234,0.02379,0.01615,0.1897,...,35.19,77.98,455.7,0.14990,0.13980,0.11250,0.06136,0.3409,0.08147,0
4,858970,10.170,14.88,64.55,311.9,0.11340,0.08061,0.01084,0.01290,0.2743,...,17.45,69.86,368.6,0.12750,0.09866,0.02168,0.02579,0.3557,0.08020,0


In [3]:
#Splitting the data into features and target
X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]

X_test = test_data.iloc[:,:-1]
y_test = test_data.iloc[:,-1]
X_train.head()


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,859471,9.029,17.33,58.79,250.5,0.10660,0.14130,0.31300,0.04375,0.2111,...,10.31,22.65,65.50,324.7,0.14820,0.43650,1.25200,0.17500,0.4228,0.11750
1,873593,21.090,26.57,142.70,1311.0,0.11410,0.28320,0.24870,0.14960,0.2395,...,26.68,33.48,176.50,2089.0,0.14910,0.75840,0.67800,0.29030,0.4098,0.12840
2,859196,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,...,10.01,19.23,65.59,310.1,0.09836,0.16780,0.13970,0.05087,0.3282,0.08490
3,88466802,10.650,25.22,68.01,347.0,0.09657,0.07234,0.02379,0.01615,0.1897,...,12.25,35.19,77.98,455.7,0.14990,0.13980,0.11250,0.06136,0.3409,0.08147
4,858970,10.170,14.88,64.55,311.9,0.11340,0.08061,0.01084,0.01290,0.2743,...,11.02,17.45,69.86,368.6,0.12750,0.09866,0.02168,0.02579,0.3557,0.08020


In [4]:
#Scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Converting the data into dataframes
X_train = pd.DataFrame(X_train, columns = train_data.columns[:-1])
X_test = pd.DataFrame(X_test, columns = test_data.columns[:-1])


In [5]:
import os
from dotenv import load_dotenv
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM


load_dotenv()  # Load environment variables from .env file
token = os.getenv("HF_TOKEN")

# Load the Mistral model and tokenizer from Hugging Face
model_name = "mistralai/Mistral-7B-v0.1"

# Ensure compatibility by using AutoTokenizer and AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_name, num_labels=2,token=token,device_map="auto")

# Ensure pad_token_id is set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [ ]:
# Function to create the prompt for in-context learning
# Create a new function for constructing prompts
def create_prompt(X_train, y_train, test_row):
    """
    This function generates a prompt for the model by providing examples
    of input features and their corresponding labels from the training set, followed by
    the test instance for which we want the model to predict the label.
    """
    prompt = ""

    # Add the test instance for prediction
    test_features = ", ".join([f"{col}={test_row[col]}" for col in X_train.columns])
    prompt += f"Input: {test_features} -> Output: ? \n"  

    # Add examples from the training set to the prompt
    for i, train_row in X_train.iterrows():
        input_features = ", ".join([f"{col}={train_row[col]}" for col in X_train.columns])
        label = y_train.iloc[i]
        prompt += f"Input: {input_features} -> Output: {label}\n"
    
    return prompt

# Example: Create a prompt for the first test instance
prompt = create_prompt(X_train, y_train, X_test.iloc[0])  # Correct usage of .iloc
print(prompt)



In [ ]:
from transformers import set_seed
# Set the seed for reproducibility
set_seed(42)
# Tokenize the prompt with truncation
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=300)
# Generate prediction
input_length = inputs['input_ids'].shape[1]
print(input_length)
#output = model.generate(**inputs, max_new_tokens=10)  # Control how many tokens are generated
output = model.generate(**inputs,min_length=input_length, max_new_tokens= 400, do_sample=True, temperature=0.7, top_k=50, pad_token_id=tokenizer.pad_token_id)


# Decode the generated prediction
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)




In [ ]:

def extract_prediction(generated_text, prompt):
    """
    Extracts the predicted label from the generated text.
    Assumes the generated output follows the pattern "Input: <features> -> Output: <prediction>".
    """
    try:
        # Use regular expression to find the value after "output="
        #match = re.search(r'(Output|class|target)\s*[:=]\s*([-+]?\d*\.\d+|\d+)', generated_text)
        match = re.search(r'Output\s*:\s*([-+]?\d*\.\d+|\d+)', generated_text)
        if match:
            prediction = match.group(1)
            print(prediction)
            return (int(round(float(prediction))) if prediction.replace('.', '', 1).isdigit() else None)
        else:
            return None
    except Exception as e:
        print(f"Error extracting prediction: {e}")
        return None

In [ ]:
from transformers import set_seed
# Function to generate predictions for a batch of test data
def in_context_learning(X_train, y_train, X_test, model, tokenizer):
    """
    Generates predictions for the test set using in-context learning.
    """
    # Set the seed for reproducibility
    # set_seed(42)
    
    
    predictions = []
    temp = []
    for _, test_row in X_test.iterrows():
        prompt = create_prompt(X_train, y_train, test_row)
        
        # Tokenize the prompt and generate output
        inputs = tokenizer(prompt, return_tensors="pt",truncation=True, max_length=300)
        input_length = inputs['input_ids'].shape[1]
        output = model.generate(**inputs,min_length=input_length, max_new_tokens=200, do_sample=True, temperature=0.7, top_k=50,) # Adjust parameters as needed, these are working parameters
        
        # Decode the generated output
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        temp.append(generated_text)
        # Extract the prediction from the generated text
        predicted_label = extract_prediction(generated_text, prompt)
        predictions.append(predicted_label)
    
    return predictions, temp




# Generate predictions for the test set
batch_size = 8  # Adjust batch size as needed
predictions = []
temp = []
pd.DataFrame(predictions).to_csv('results/mistralmain.csv', index=False)
pd.DataFrame(temp).to_csv('results/textmistral.csv', index=False)

for i in range(0, len(X_test), batch_size):
    X_batch = X_test[i:i + batch_size]
    batch_predictions, textsp = in_context_learning(X_train, y_train, X_batch, model, tokenizer)
    predictions.extend(batch_predictions)
    temp.extend(textsp)
    
# Append the predictions to the results folder in mistral.csv file
pd.DataFrame(predictions).to_csv('results/mistralmain.csv', mode='a', header=False, index=False)

# Append the text to the results folder in textmistral.csv file
pd.DataFrame(temp).to_csv('results/textmistral.csv', mode='a', header=False, index=False)






## This part of the code might throw an error if the predictions donot match the specified format in this case manual work to scrape the predictions must be performed from the generated csv 

In [ ]:
# Evaluate the Mistral model with confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report

# Calculate confusion matrix
confusion = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")

# Display confusion matrix
print(confusion)

# Calculate classification report
report = classification_report(y_test, predictions)
print("\nClassification Report:")
# Display classification report
print(report)

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
